<a href="https://colab.research.google.com/github/jyotidabass/Binary_tongue_classification/blob/main/Binary_tongue_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content/drive/MyDrive/Two class

/content/drive/MyDrive/Two class


In [3]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from tensorflow.keras.applications import ResNet50, MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools


%matplotlib inline

In [4]:
#Transfer 'jpg' images to an array IMG
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".png":
            img = read(PATH)
           
            img = cv2.resize(img, (RESIZE,RESIZE))
           
            IMG.append(np.array(img))
    return IMG

Kapha_train = np.array(Dataset_loader('/content/drive/MyDrive/Two class/Train/Tongue Image - Kapha',224))
Vata_train = np.array(Dataset_loader('/content/drive/MyDrive/Two class/Train/Tongue Image - Vata',224))
Kapha_test = np.array(Dataset_loader('/content/drive/MyDrive/Two class/Train/Tongue Image - Kapha',224))
Vata_test = np.array(Dataset_loader('/content/drive/MyDrive/Two class/Train/Tongue Image - Vata',224))

100%|██████████| 189/189 [00:00<00:00, 10263.65it/s]


In [5]:
# Skin Cancer: Malignant vs. Benign
# Create labels
Kapha_train_label = np.zeros(len(Kapha_train))
Vata_train_label = np.ones(len(Vata_train))
Kapha_test_label = np.zeros(len(Kapha_test))
Vata_test_label = np.ones(len(Vata_test))

# Merge data 
X_train = np.concatenate((Kapha_train, Vata_train), axis = 0)
Y_train = np.concatenate((Kapha_train_label, Vata_train_label), axis = 0)
X_test = np.concatenate((Kapha_test, Vata_test), axis = 0)
Y_test = np.concatenate((Kapha_test_label, Vata_test_label), axis = 0)

# Shuffle train data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

# Shuffle test data
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

# To categorical
Y_train = to_categorical(Y_train, num_classes= 2)
Y_test = to_categorical(Y_test, num_classes= 2)

In [6]:
x_train, x_val, y_train, y_val = train_test_split(
    X_train, Y_train, 
    test_size=0.2, 
    random_state=11
)

In [7]:
BATCH_SIZE = 16

# Using original generator
train_generator = ImageDataGenerator(
        zoom_range=2,  # set range for random zoom
        rotation_range = 90,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )

In [8]:
def build_model(backbone, lr=1e-4):
    model = Sequential()
    model.add(backbone)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(2, activation='softmax'))
    
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=lr),
        metrics=['accuracy']
    )
    
    return model

In [9]:
K.clear_session()
gc.collect()

resnet = DenseNet201(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)



model = build_model(resnet ,lr = 1e-4)
model.summary()

74850304/74836368 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                                 
 global_average_pooling2d (G  (None, 1920)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1920)              0         
                                                                 
 batch_normalization (BatchN  (None, 1920)             7680      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 2)                 3842      
                                                         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [10]:
#Learning Rate Reducer
learn_control = ReduceLROnPlateau(monitor='val_acc', patience=5,
                                  verbose=1,factor=0.2, min_lr=1e-7)

# Checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [13]:

# Import Statements
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten

#Initialising CNN
classifier = Sequential()

#Adding Convolution Layer 1

classifier.add(Convolution2D(32,3,3,input_shape=(64,64,3),activation='relu'))
#Pooling
classifier.add(MaxPool2D(pool_size=(2,2)))

#Adding Convolution Layer 2

classifier.add(Convolution2D(32,3,3,activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))

#Flatten
classifier.add(Flatten())

#Full Connection
classifier.add(Dense(128,activation='relu'))
#classifier.add(Dense(output_dim=64,activation='relu'))
classifier.add(Dense(1,activation='sigmoid'))

#compile
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#Fitting the model to images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen= ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Two class/Train',
                                                    target_size=(64,64),
                                                    batch_size=32,
                                                    class_mode='binary')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Two class/Train',
                                                        target_size=(64,64),
                                                        batch_size=32,
                                                        class_mode='binary')

classifier.fit_generator(training_set,
                        steps_per_epoch=10,
                        epochs=5,
                        validation_data=test_set,
                        validation_steps=2,
                        workers=2
                        )

Found 426 images belonging to 2 classes.
Found 426 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
10/10 [==============================] - 58s 6s/step - loss: 0.6877 - accuracy: 0.5369 - val_loss: 0.6963 - val_accuracy: 0.5312
Epoch 2/5
10/10 [==============================] - 58s 5s/step - loss: 0.6831 - accuracy: 0.5738 - val_loss: 0.6834 - val_accuracy: 0.5469
Epoch 3/5
10/10 [==============================] - 51s 4s/step - loss: 0.6843 - accuracy: 0.5705 - val_loss: 0.6861 - val_accuracy: 0.5625
Epoch 4/5
10/10 [==============================] - 46s 5s/step - loss: 0.6821 - accuracy: 0.5772 - val_loss: 0.6568 - val_accuracy: 0.6406
Epoch 5/5
10/10 [==============================] - 44s 4s/step - loss: 0.6765 - accuracy: 0.5844 - val_loss: 0.6871 - val_accuracy: 0.5625
